In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Sep  3 15:31:18 2018

@author: winstonlin
"""
import os
import numpy as np
from scipy.io import savemat
import subprocess
import json
from glob import glob
from pathlib import Path
import shutil
from tqdm import tqdm

In [2]:
root = '/home/a/Desktop/MSP-Podcast/pipeline/'

output_location = 'INPUTS_OUTPUTS/Outputs/'

In [3]:
OPENSMILE_PATH = '/home/a/Desktop/MSP-Podcast/pipeline/emotion_retrieval_final/opensmile/opensmile/build/progsrc/smilextract/SMILExtract'

In [4]:
def SmileExtract(input_path, out_path, audio_files, feature_type, desc_type = 'lld'):
    ERROR_record = ''
    for filename, f_info in tqdm(audio_files.items()):
        f_path = f_info['filepaths']['wav']
        if '.wav' in f_path:
            input_file = f_path
            output_file = os.path.join(out_path, filename + '.arff')
            f_info['filepaths']['opensmile_'+desc_type] = output_file

            if desc_type == 'lld':

                cmd = OPENSMILE_PATH+' -l 1 -C opensmile-2.3.0_lld/IS13_ComParE.conf -I ' + input_file + ' -O ' + output_file
            elif desc_type == 'hld':
                cmd = OPENSMILE_PATH+' -l 1 -C opensmile-2.3.0_hld/IS13_ComParE.conf -I ' + input_file + ' -O ' + output_file
            else:
                raise ValueError('Unsupport desc_type Type!')
            try:
                subprocess.call(cmd, shell=True)
            except:
                ERROR_record += 'Error: '+fileNames[i]+'\n'

        else:
            ERROR_record += 'Source not WAV file: ' +fileNames[i]+'\n'
    print(ERROR_record)

In [5]:

    
# feature_arff to feature_mat
def TryToFloat(single_data):
    try:
        return float(single_data)
    except:
        return None

def LoadFeature(filename):
    content = open(filename, 'r').read()
    data = content.split('@data\n')[1].split('\n')
    data = filter(None, data)
    feature = [[TryToFloat(data_split) for data_split in d.split(',') \
                if TryToFloat(data_split)!=None] for d in data]
    return feature

In [6]:
with open(os.path.join(os.path.join(root, output_location), "Short_files.json"), "r") as openfile:
    audio_files = json.load(openfile)

In [7]:

####################### eGeMAPS-LLD ####################
feat = 'OpenSmile_lld_IS13ComParE'  # Dim=130 LLDs

# Part 1: OpenSmile => Arff File
input_path = os.path.join(root, output_location, "Short_split_file")
out_path = os.path.join(root, output_location, "Short_split_"+feat)
if not os.path.exists(out_path):
    os.mkdir(out_path)

print("Running openSMILE lld  extractor")
SmileExtract(input_path, out_path, audio_files, feature_type='IS13ComParE', desc_type='lld')

  1%|          | 3/499 [00:00<00:20, 24.15it/s]

Running openSMILE lld  extractor


100%|██████████| 499/499 [00:18<00:00, 26.69it/s]

In [8]:
feat = 'OpenSmile_hld_IS13ComParE' # Dim=6373 HLDs
input_path = os.path.join(root, output_location, "Short_split_file")
out_path = os.path.join(root, output_location, "Short_split_"+feat)
if not os.path.exists(out_path):
    os.mkdir(out_path)
print("Running openSMILE hld extractor")
SmileExtract(input_path, out_path, audio_files, feature_type='IS13ComParE', desc_type='hld') 

  1%|          | 3/499 [00:00<00:22, 22.08it/s]

Running openSMILE hld extractor


100%|██████████| 499/499 [00:21<00:00, 23.07it/s]

In [9]:
json_object = json.dumps(audio_files, indent=4)
with open(os.path.join(root, output_location, "Short_files.json"), "w") as outfile:
    outfile.write(json_object)

In [10]:
end of file

SyntaxError: invalid syntax (<ipython-input-10-67aa0ebee22b>, line 1)

In [ ]:
# for filename in glob(os.path.join(out_path,'*.arff')):
#     basename = os.path.basename(filename).split('.')[0]
#     data = LoadFeature(filename)
#     # data = np.array(data)
#     audio_files[basename]['opensmile_lld'] = data


In [ ]:
# shutil.rmtree(out_path)

In [ ]:
# savemat(output_file, {'Audio_data':data})

In [ ]:
for filename in glob(os.path.join(out_path,'*.arff')):
    basename = os.path.basename(filename).split('.')[0]
    data = LoadFeature(filename)
    # data = np.array(data)
    audio_files[basename]['opensmile_hld'] = data

In [ ]:
shutil.rmtree(out_path)

In [ ]:
json_object = json.dumps(audio_files, indent=4)
with open(os.path.join(root, output_location, "Short_files.json"), "w") as outfile:
    outfile.write(json_object)

In [ ]:
#combines 
import os
import numpy as np
from scipy.io import wavfile
import textgrid
import subprocess
import shutil
from glob import glob
import shutil
from datetime import datetime, timedelta
import json



def parse_exist_files(root_path):
    FileNames = []
    for dirPath, dirNames, fileNames in os.walk(root_path):
        fileNames = sorted(fileNames)
        for i in range(len(fileNames)):
            fname = fileNames[i]
            fname = fname.replace('.wav','')
            fname = fname.replace('.txt','')
            fname = fname.replace('.TextGrid','')
            FileNames.append(fname)
    FileNames = np.unique(np.array(FileNames)).tolist()  
    return FileNames
##############################################################################
def trim_time(time_d):
    time_d = str(time_d)
    if '.' in time_d:
        if len(time_d.split('.')[-1])>2:
            trim_digits = len(time_d.split('.')[-1])-2
            time_d = time_d[:-trim_digits]
    else:
        time_d += '.00'
    return time_d


In [ ]:




    
    
    
    
# Parameters
output_root = '../INPUTS_OUTPUTS/Outputs'


output_short_file = os.path.abspath(os.path.join(output_root, 'Short_split_file'))


reseg_output_file = os.path.join(output_root,'Long_reseg_file')
mfa_corpus_root = os.path.join(output_root,'mfa_corpus/speaker1')
mfa_aligned_root = os.path.join(output_root,'mfa_align_rsl/speaker1')
seg_threshold = 0.3 # in secs


with open(os.path.join(output_root, "Short_files.json"), "r") as openfile:
    short_files = json.load(openfile)
    
with open(os.path.join(output_root, "Long_files.json"), "r") as openfile:
    long_files = json.load(openfile)

os.mkdir(reseg_output_file)

In [ ]:
# get info for any non-align files
missing_files = list(set(parse_exist_files(mfa_corpus_root))-set(parse_exist_files(mfa_aligned_root)))
missing_files = sorted(missing_files)
print("*****NOTE: Missing Aligned Files= "+str(len(missing_files))+" *****")


def save_segment_text(out_name, text):
    with open(out_name,'w', encoding='utf8') as wf:
        wf.write(text)
        wf.close()
        
        
def read_segment_text(file_name):
    with open(file_name) as f:
        lines = f.read()
    return lines

In [ ]:
def trim_time(time_d):
    time_d = str(time_d)
    if '.' in time_d:
        if len(time_d.split('.')[-1])>2:
            trim_digits = len(time_d.split('.')[-1])-2
            time_d = time_d[:-trim_digits]
    else:
        time_d += '.00'
    return time_d

In [ ]:
def timestr_to_seconds(time_str):
    ftr = [3600,60,1]

    seconds = sum([a*b for a,b in zip(ftr, map(int,time_str.split('.')[0].split(':')))])
    remainder = float(time_str.split('.')[1])/100
    total_time = seconds+ remainder
    return total_time

In [ ]:
# main function
aligned_files_all = sorted(os.listdir(mfa_aligned_root))
for filepath in glob(os.path.join(mfa_aligned_root,'*.TextGrid')):
    
    filename = os.path.basename(filepath).split('.')[0]
    align_rsl = textgrid.TextGrid.fromFile(filepath)[0]
    break

In [ ]:
start_time = long_files[filename]['start']
start_time

In [ ]:
start_time2 = "0:00:10.20"

In [ ]:
start_time = timestr_to_seconds(start_time) + timestr_to_seconds(start_time2)
trim_time(timedelta(seconds = start_time2))

In [ ]:







    
    
    # obtain split timestamps
    seg_times = []
    seg_asr = ["",]
    for j in range(len(align_rsl)):
        word = align_rsl[j].mark
        word_t_start = align_rsl[j].minTime
        word_t_end = align_rsl[j].maxTime
        duration = word_t_end - word_t_start
        if word=="" and duration>=seg_threshold:
            seg_times.append(str((word_t_start+word_t_end)/2))
            seg_asr.append("")
        elif word == "":
            continue
        else:
            seg_asr[-1] += word + ' ' 